In [7]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import skimage
from PIL import Image
import os
import cv2
from skimage import transform

In [45]:


def extract_hog_features(img):
    # image = skimage.io.imread('teste/treinamento/000001.jpg',as_gray=True)
    img = skimage.transform.resize(img, (128,64))
    # print(len(img[1]))
    mag = []
    theta = []

    for i in range(128):
        magnitudeArray = []
        angleArray = []

        for j in range(64):
            # Condition for axis 0
            if j-1 <= 0 or j+1 >= 64:
                if j-1 <= 0:
                    # Condition if the first element
                    Gx = img[i, j+1] - 0
                elif j + 1 >= 64:
                    Gx = 0 - img[i, j-1]
            # Condition for the first element
            else:
                Gx = img[i, j+1] - img[i, j-1]

            # Condition for axis 1
            if i-1 <= 0 or i+1 >= 128:
                if i-1 <= 0:
                    Gy = 0 - img[i+1, j]
                elif i +1 >= 128:
                    Gy = img[i-1, j] - 0
            else:
                # print(i, j)
                Gy = img[i-1, j] - img[i+1, j]

            # Calculating magnitude
            # print(Gx)
            magnitude = math.sqrt(pow(Gx, 2) + pow(Gy, 2))
            magnitudeArray.append(round(magnitude, 9))

            # Calculating angle
            if Gx == 0:
                angle = np.degrees(0.0)
            else:
                angle = np.degrees(np.abs(np.arctan(Gy / Gx)))
            angleArray.append(round(angle, 9))

        mag.append(magnitudeArray)
        theta.append(angleArray)

    mag = np.array(mag)
    theta = np.array(theta)

    row, col = mag.shape
    features = []

    for i in range(0, row, 8):
        for j in range(0, col, 8):
            curr_mag = mag[i:i+8, j:j+8]
            curr_theta = theta[i:i+8, j:j+8]
            window_row, window_col = curr_mag.shape
            histogram = np.zeros(9)

            for k in range (window_row):
                for l in range (window_col):
                    first_bin = int(curr_theta[k, l] / 20)
                    second_bin = (math.ceil(curr_theta[k, l] / 20)) % 9

                    if first_bin == second_bin:
                        histogram[first_bin] += curr_mag[k, l]
                    else:
                        histogram[first_bin] += curr_mag[k, l] * (1 - (curr_theta[k, l] - first_bin * 20) / 20)
                        histogram[second_bin] += curr_mag[k, l] * ((curr_theta[k, l] - first_bin * 20) / 20)

            features += histogram.tolist()
    #plt.imshow(theta, cmap='gray')
    #plt.axis('off')
    #plt.show()
    #plt.figure(figsize=(10,10))
    #plt.imshow(mag, cmap='gray')
    #plt.axis('off')
    #plt.show()
    return features


In [43]:
number_of_bins = 9
step_size = 180 / number_of_bins

def calculate_j(angle):
    temp = (angle / step_size) - 0.5
    j = math.floor(temp)
    return j

def calculate_Cj(j):
    Cj = step_size * (j + 0.5)
    return round(Cj, 9)

def calculate_value_j(magnitude, angle, j):
    Cj = calculate_Cj(j+1)
    Vj = magnitude * ((Cj - angle) / step_size)
    return round(Vj, 9)

def extract_hog_features(img):
    img = skimage.transform.resize(img, (128,64))
    # print(len(img[1]))
    mag = []
    theta = []

    for i in range(128):
        magnitudeArray = []
        angleArray = []

        for j in range(64):
            # Condition for axis 0
            if j-1 <= 0 or j+1 >= 64:
                if j-1 <= 0:
                    # Condition if the first element
                    Gx = img[i, j+1] - 0
                elif j + 1 >= 64:
                    Gx = 0 - img[i, j-1]
            # Condition for the first element
            else:
                Gx = img[i, j+1] - img[i, j-1]

            # Condition for axis 1
            if i-1 <= 0 or i+1 >= 128:
                if i-1 <= 0:
                    Gy = 0 - img[i+1, j]
                elif i +1 >= 128:
                    Gy = img[i-1, j] - 0
            else:
                # print(i, j)
                Gy = img[i-1, j] - img[i+1, j]

            # Calculating magnitude
            # print(Gx)
            magnitude = math.sqrt(pow(Gx, 2) + pow(Gy, 2))
            magnitudeArray.append(round(magnitude, 9))

            # Calculating angle
            if Gx == 0:
                angle = np.degrees(0.0)
            else:
                angle = np.degrees(np.abs(np.arctan(Gy / Gx)))
            angleArray.append(round(angle, 9))

        mag.append(magnitudeArray)
        theta.append(angleArray)

    mag = np.array(mag)
    theta = np.array(theta)

    row, col = mag.shape
    histogram = []

    for i in range(0, 128, 8):
        for j in range(0, 64, 8):
            curr_mag = mag[i:i+8, j:j+8]
            curr_theta = theta[i:i+8, j:j+8]

            temp = []
            for k in range(len(curr_mag)):
                for l in range(len(curr_mag[0])):
                    bins = [0.0 for x in range(9)]
                    value_j = calculate_j(curr_theta[k, l])
                    Vj = calculate_value_j(curr_mag[k, l], curr_theta[k, l], value_j)
                    Vj_1 = curr_mag[k, l] - Vj
                    bins[value_j] += Vj
                    bins[value_j+1] += Vj_1
                    bins = [round(x, 9) for x in bins]
                    temp.extend(bins)

            histogram.extend(temp)

    epsilon = 1e-05
    features = []

    for i in range(0, len(histogram)-1, 2):
        for j in range(0, len(histogram)-1, 2):
            values = np.array([[histogram[i], histogram[i+1]], [histogram[j], histogram[j+1]]])
            final_vector = values.flatten()

            k = round(math.sqrt(np.sum(np.square(final_vector))), 9)
            final_vector = [round(x / (k + epsilon), 9) for x in final_vector]

            features.extend(final_vector)
    
    return features

In [9]:
labels_dict = {}
f = open("labels.csv", "r")
for line in f.read().split("\n"):
    if(len(line) > 1):
        key,value = line.split(",")
        labels_dict[key] = value
f.close()


In [10]:
img = skimage.io.imread("dados/CAPTCHA-10k/treinamento/000001.jpg")
img.shape

(50, 180, 3)

In [46]:
def crop_images(folder, csv_name):
    features = []
    
    for filename in os.listdir(folder):
        image_path = os.path.join(folder, filename)
        image = skimage.io.imread(image_path,as_gray=True)  # Specify as_gray=True if you want a grayscale image
        key = filename.replace(".jpg", "")
        label = labels_dict[key]

        rows_per_part = image.shape[1] // 6
        # features = []
        
        for i in range(6):
            # Crop and resize each part to the specified size (50, 30)
            part = image[:, i * rows_per_part:(i + 1) * rows_per_part]
            part_resized = cv2.resize(part, (30, 50))
            crop = extract_hog_features(part_resized)
            crop = [0.0 if x is None else x for x in crop]
            features.append([label[i]] + crop)
        # features_vector.append(features)
    # features_vector = np.array(features_vector)
    pd.DataFrame(features).to_csv(csv_name, index=False, header=False)

teste_folder = "teste/treinamento/"
crop_images(teste_folder, "teste.csv")

In [47]:
test = pd.read_csv("teste.csv", header=None)

,0,1,2,3,4,5,6,7,8,9,...,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152
0,O,2.340983,2.182023,2.033623,4.980075,7.701241,0.938231,0.0,0.0,0.0,...,0.0,5.133949,3.766645,2.618541,4.085227,4.990982,0.625508,0.0,0.0,0.0
1,Q,0.444544,5.759552,4.518512,3.408999,8.922685,3.362940,0.0,0.0,0.0,...,0.0,8.839180,4.622580,5.430759,4.296754,2.455837,0.256690,0.0,0.0,0.0
2,Z,7.002985,5.641532,4.708476,2.193626,7.155097,5.422312,0.0,0.0,0.0,...,0.0,5.132459,3.380919,3.306703,2.495239,4.733946,2.488081,0.0,0.0,0.0
3,S,11.047492,1.102577,4.663452,1.837309,6.607350,5.812918,0.0,0.0,0.0,...,0.0,4.635672,3.042182,1.882996,0.696857,4.588693,2.344154,0.0,0.0,0.0
4,L,11.611124,0.389036,4.279395,1.452800,6.174972,5.815905,0.0,0.0,0.0,...,0.0,5.469354,1.622888,1.426358,0.685116,4.035807,3.424448,0.0,0.0,0.0
5,4,2.780600,2.234006,8.059916,6.180392,10.630579,5.446986,0.0,0.0,0.0,...,0.0,5.440980,1.921694,1.310090,0.580462,3.915844,3.496472,0.0,0.0,0.0
6,G,1.292990,1.623761,1.297717,3.868919,5.283737,0.537604,0.0,0.0,0.0,...,0.0,6.025135,1.841519,1.388859,0.639141,4.253171,3.272436,0.0,0.0,0.0
7,S,9.622493,1.896552,5.446417,4.225570,7.410270,6.156201,0.0,0.0,0.0,...,0.0,6.857387,0.622300,1.958630,0.685056,3.893368,3.415474,0.0,0.0,0.0
8,T,11.316364,0.650606,4.533081,1.706302,6.867089,5.482563,0.0,0.0,0.0,...,0.0,6.966732,0.221958,1.080685,0.414884,3.774472,3.309913,0.0,0.0,0.0
9,2,11.486758,0.571862,4.389772,1.578529,6.660570,5.352872,0.0,0.0,0.0,...,0.0,7.039318,0.148470,1.120802,0.389572,3.658977,3.313586,0.0,0.0,0.0


In [48]:
train_folder = "dados/CAPTCHA-10k/treinamento"
validation_folder = "dados/CAPTCHA-10k/validacao"
test_folder = "dados/CAPTCHA-10k/teste"

crop_images(train_folder, "features_treinamento.csv")
crop_images(validation_folder, "features_validacao.csv")
crop_images(test_folder, "features_teste.csv")

In [13]:
from sklearn.preprocessing import StandardScaler

df_train = pd.read_csv("features_treinamento.csv", header=None)
X_train = df_train.iloc[:, 1:]
y_train = df_train.iloc[:, 0].values

labels = sorted(set(y_train))
dict_transform = {label: i for i, label in enumerate(labels)}

y_train = np.array([dict_transform[label] for label in y_train])
X_train = (X_train - X_train.mean()) / X_train.std()
X_train = X_train.fillna(0)

df_validation = pd.read_csv("features_validacao.csv", header=None)
X_validation = df_validation.iloc[:, 1:]
y_validation = df_validation.iloc[:, 0].values

y_validation = np.array([dict_transform[label] for label in y_validation])
X_validation = (X_validation - X_validation.mean()) / X_validation.std()
X_validation = X_validation.fillna(0)

df_test = pd.read_csv("features_teste.csv", header=None)
X_test = df_test.iloc[:, 1:]
y_test = df_test.iloc[:, 0].values

y_test = np.array([dict_transform[label] for label in y_test])
X_test = (X_test - X_test.mean()) / X_test.std()
X_test = X_test.fillna(0)


In [14]:
# Classificar os modelos SVM
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Split the data into train and test
model = SVC(random_state=2)
model.fit(X_train, y_train)

# Predict on train and test
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_validation)

# Print the results
print("Accuracy on train set: ", metrics.accuracy_score(y_train, y_pred_train))
print("Accuracy on test set: ", metrics.accuracy_score(y_validation, y_pred_test))




Accuracy on train set:  0.9786666666666667
Accuracy on test set:  0.0295


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
import sklearn.metrics as metrics

models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1500, solver='newton-cg', penalty='l2'),
    'KNN': KNeighborsClassifier(n_neighbors=10, weights='distance', algorithm='kd_tree', metric='minkowski', p=1),
    'Random Forest': RandomForestClassifier(criterion='gini', n_estimators=50, random_state=42),
    'Decision Tree': DecisionTreeClassifier(criterion='gini', random_state=42),
    'SVM': SVC(random_state=42),
    'SGDC': SGDClassifier(loss='perceptron', random_state=42)
}

for model_name, model in models.items():
    print(model_name)
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_validation)
    print("Accuracy on train set: ", metrics.accuracy_score(y_train, y_pred_train))
    print("Accuracy on test set: ", metrics.accuracy_score(y_validation, y_pred_test))
    print("\n")


Logistic Regression
Accuracy on train set:  0.9484166666666667
Accuracy on test set:  0.7133333333333334


KNN
Accuracy on train set:  1.0
Accuracy on test set:  0.6758333333333333


Random Forest
Accuracy on train set:  1.0
Accuracy on test set:  0.7313333333333333


Decision Tree
Accuracy on train set:  1.0
Accuracy on test set:  0.4615


SVM
Accuracy on train set:  0.9786666666666667
Accuracy on test set:  0.8261666666666667


SGDC
Accuracy on train set:  0.785875
Accuracy on test set:  0.7173333333333334




In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
import sklearn.metrics as metrics

models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1500, solver='newton-cg', penalty='l2'),
    'KNN': KNeighborsClassifier(n_neighbors=10, weights='distance', algorithm='kd_tree', metric='minkowski', p=1),
    'Random Forest': RandomForestClassifier(criterion='gini', n_estimators=50, random_state=42),
    'Decision Tree': DecisionTreeClassifier(criterion='gini', random_state=42),
    'SVM': SVC(random_state=42),
    'SGDC': SGDClassifier(loss='perceptron', random_state=42)
}

for model_name, model in models.items():
    print(model_name)
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    print("Accuracy on train set: ", metrics.accuracy_score(y_train, y_pred_train))
    print("Accuracy on test set: ", metrics.accuracy_score(y_test, y_pred_test))
    print("\n")


Logistic Regression
Accuracy on train set:  0.9484166666666667
Accuracy on test set:  0.716


KNN
Accuracy on train set:  1.0
Accuracy on test set:  0.6698333333333333


Random Forest
Accuracy on train set:  1.0
Accuracy on test set:  0.7316666666666667


Decision Tree
Accuracy on train set:  1.0
Accuracy on test set:  0.4736666666666667


SVM
Accuracy on train set:  0.9786666666666667
Accuracy on test set:  0.8295


SGDC
Accuracy on train set:  0.785875
Accuracy on test set:  0.7165


